In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250509'

## Import Connected TV TCs and add other device columns

### Group by main and sub categories to create `df_dev_component`

#### test cases will be joined by ", "

In [ ]:
# Initial Setup for merge later

df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").reset_index(drop=True)
df_connectedTV_raw = df_connectedTV_raw.rename(columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"})
available_devices = ['Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web']

df_K4_connectedTV = df_connectedTV_raw.fillna(0).copy()
df_K4_connectedTV["Web"] = False
df_K4_connectedTV["Apple Mobile"] = False
df_K4_connectedTV["Android Mobile"] = False
df_K4_connectedTV["Smart TV"] = True
df_K4_connectedTV["Vizio TV"] = True
df_K4_connectedTV["project_name"] = "KOCOWA 4.0"

# Convert "UNTESTED" to True boolean flags
for index, row in df_K4_connectedTV.iterrows():
    for i in range(6, 10):  # Columns from index 6 to 9
        if row.iloc[i] == "UNTESTED":
            df_K4_connectedTV.iat[index, i] = True

df_K4_connectedTV = df_K4_connectedTV.rename(columns={
    "대분류": "main_category",
    "중분류": "sub_category",
    "소분류": "component",
    "Section": "scenario",
    "테스트 항목": "test_case",
    "기대결과": "expected_result"})

device_columns = ['Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']
col_orders = ['project_name','main_category', 'component', 'test_case', 'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web','Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV']

# If component = 0, then fill it with sub_category
for index, row in df_K4_connectedTV.iterrows():
    main_category = row.main_category
    component = row.component

    if main_category == "Contents":
        main_category_rename = "Content Details"
        df_K4_connectedTV.at[index, "main_category"] = main_category_rename

    # Fill missing sub categories and components
    if component == 0:
        if row.sub_category == 0:
            component_rename = row.main_category
            df_K4_connectedTV.at[index, "sub_category"] = component_rename
        else:
            component_rename = row.sub_category
        df_K4_connectedTV.at[index, "component"] = component_rename

# Rule: If the name of the component is the same as main category, use "General Rule" tag
for index, row in df_K4_connectedTV.iterrows():
    main_category = row.main_category
    component = row.component

    if component.strip().lower() == main_category.strip().lower():
        component_rename = "General Rule"
        df_K4_connectedTV.at[index, "component"] = component_rename
    
    # Rule for 4.0 Project: if main category is Content Details and My KOCOWA, True for all web and mobile platforms
    if main_category == "My KOCOWA Page":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
    elif main_category == "Content Details":
        df_K4_connectedTV.at[index, "Web"] = True
        df_K4_connectedTV.at[index, "Apple Mobile"] = True
        df_K4_connectedTV.at[index, "Android Mobile"] = True
        
# Grouping the dataframe by main_category, sub_category, and component
# Aggregating test_case by joining them with ", " and taking the maximum value for device_columns
df_dev_component = (
    df_K4_connectedTV
    .groupby(['project_name','main_category', 'component'])
    .agg({
        'test_case': lambda x: '\n'.join(x),  # Combine test cases into a single string
        **{col: 'max' for col in device_columns}  # Take the maximum value for each device column
    })
    .reset_index()  # Reset the index to make it a regular dataframe
).sort_values(by = ["main_category", "component"]).copy()
        
df_dev_component_cleaned = df_dev_component[df_dev_component.component != "General Rule"][col_orders].copy()

In [11]:
len(list(df_dev_component.main_category.unique()))

21

In [13]:
df_dev_component_cleaned

,project_name,main_category,component,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
0,KOCOWA 4.0,Content Details,Bookmarked Button,Bookmarked Button 확인\nBookmarked Button icon 확...,True,True,True,True,False,False,False,True,True
1,KOCOWA 4.0,Content Details,Cast Section,title text 확인\nContent Cast 확인,True,True,True,True,False,False,False,True,True
2,KOCOWA 4.0,Content Details,Continue Watching Button,Continue Watching Button 확인\nContinue Watching...,True,True,True,True,False,False,False,True,True
3,KOCOWA 4.0,Content Details,Details Section,Content Released Year 확인\nGenre text 확인\nDivis...,True,True,True,True,False,False,False,True,True
4,KOCOWA 4.0,Content Details,Director Section,title text 확인\nContent Cast 확인,True,True,True,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,KOCOWA 4.0,VARIETY Channel Page,Sorting POP UP,Default Setting 확인\nCheck icon 확인\n리모컨 아래 방향키 ...,True,True,True,True,False,False,False,True,True
229,KOCOWA 4.0,VARIETY Channel Page,Top Area,Division Line 확인,True,True,True,True,False,False,False,True,True
230,KOCOWA 4.0,VARIETY Channel Page,Update Soon Screen,Speech Bubble icon 확인\nsub text 확인,True,True,True,True,False,False,False,True,True
232,KOCOWA 4.0,WELCOME TO KOCOWA Page,Home Button,Home Button 확인\nHome Button stringset 확인\nHome...,True,True,True,True,False,False,False,True,True


### Initial export for checkup (csv and excel): processed_KOCOWA_4.0_tc_connectedTV

In [5]:
df_dev_component_cleaned.to_csv("data/processed/processed_KOCOWA_4.0_tc_connectedTV.csv", index=False)
# df_dev_component.to_excel("data/processed/processed_KOCOWA_4.0_tc_connectedTV.xlsx", index=False)